# Uses interaction to push triples to the brain

Before running, start GraphDB and make sure that there is a sandbox repository

In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Importing the chatbot utility functions

In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend

## Specifying the BRAIN

In [3]:

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

    ### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)
    
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=False)


2021-11-05 13:57:10,603 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


We will use two packages in addition to the BRAIN for:

* extracting triples and perspectives from text: cltl.triple_extraction
* for verbalising the triples returned by the BRAIN either as a result of questions or as thoughts: cltl.reply_generation

The *talk.py* script within the *intentions* module contains the functions that connect the signals in a scenario with the brain.

In [4]:
replier = LenkaReplier()

2021-11-05 13:57:18,094 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


In [10]:
test_triple = {'predicate': {'label': 'label'}, 
               'subject': {'label': 'Fred', 'type': ['agent']}, 
               'object': {'label': 'Fred', 'type': ['noun.object']}}

thoughts = talk.post_a_triple(test_triple, replier, my_brain)
print(thoughts)

{'chat': '1', 'turn': '1', 'author': 'me', 'utterance': '', 'utterance_type': <UtteranceType.STATEMENT: 0>, 'position': '', 'subject': {'label': 'Fred', 'type': ['agent']}, 'predicate': {'type': 'label'}, 'object': {'label': 'Fred', 'type': ['noun.object']}, 'context_id': '1', 'date': datetime.date(2021, 11, 5), 'place': '', 'place_id': '', 'country': '', 'region': '', 'city': '', 'objects': [], 'people': []}
2021-11-05 14:54:44,452 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: fred_label_fred [agent_->_object])
2021-11-05 14:54:44,505 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - new object 
I did not know that! I had never heard about fred before!


In [12]:
test_triple = {'predicate': {'label': 'like', 'type': ['prep']}, 
               'subject': {'label': 'Fred', 'type': ['noun.person']}, 
               'object': {'label': 'singing', 'type': ['noun.act']}}

thoughts = talk.post_a_triple(test_triple, replier, my_brain)
print(thoughts)


{'chat': '1', 'turn': '1', 'author': 'me', 'utterance': '', 'utterance_type': <UtteranceType.STATEMENT: 0>, 'position': '', 'subject': {'label': 'Fred', 'type': ['noun.person']}, 'predicate': {'type': 'like'}, 'object': {'label': 'singing', 'type': ['noun.act']}, 'context_id': '1', 'date': datetime.date(2021, 11, 5), 'place': '', 'place_id': '', 'country': '', 'region': '', 'city': '', 'objects': [], 'people': []}
2021-11-05 14:55:14,251 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: fred_like_singing [person_->_act])
2021-11-05 14:55:14,295 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Statement Novelty: 1 times, e.g. me on November,2021
2021-11-05 14:55:14,401 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Overlaps: 0 subject overlaps: e.g. '' - 1 object overlaps: e.g. me on November,2021 about lenka
I am curious. What kinds of things can like a singing like fred


In [14]:
test_triple = {'predicate': {'label': 'like', 'type': ['verb.emotion']}, 
               'subject': {'label': 'fred', 'type': ['agent']}, 
               'object': {'label': '', 'type': []}}
answer = talk.answer_a_query(test_triple, replier, my_brain)
print(answer)

{'chat': '1', 'turn': '1', 'author': 'me', 'utterance': '', 'utterance_type': <UtteranceType.QUESTION: 1>, 'position': '', 'subject': {'label': 'fred', 'type': ['agent']}, 'predicate': {'type': 'like'}, 'object': {'label': '', 'type': []}, 'context_id': '1', 'date': datetime.date(2021, 11, 5), 'place': '', 'place_id': '', 'country': '', 'region': '', 'city': '', 'objects': [], 'people': []}
2021-11-05 14:55:43,863 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: fred_like_? [agent_->_])
I have no idea.


In [9]:
test_triple = {'predicate': {'label': 'like', 'type': ['verb.emotion']}, 
               'subject': {'label': 'fred', 'type': ['agent']}, 
               'object': {'label': '', 'type': []}}
answer = talk.answer_a_query(test_triple, replier, my_brain)
print(answer)

{'chat': '1', 'turn': '1', 'author': 'me', 'utterance': '', 'utterance_type': <UtteranceType.QUESTION: 1>, 'position': '', 'subject': {'label': 'fred', 'type': ['agent']}, 'predicate': {'type': 'like'}, 'object': {'label': '', 'type': []}, 'context_id': '1', 'date': datetime.date(2021, 11, 5), 'place': '', 'place_id': '', 'country': '', 'region': '', 'city': '', 'objects': [], 'people': []}
2021-11-05 14:53:02,874 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: fred_like_? [agent_->_])
I don't know


## Standard initialisation of a scenario

In [3]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

    ### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Define the folder where the images and rdf are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  ../../data/2021-11-04-17:39:09  Created 
Directory  ../../data/2021-11-04-17:39:09/image  Created 


In [7]:
#### Small sequence to learn name of speaker
initial_prompt = f"{choice(GREETING)} {choice(ASK_NAME)} {HUMAN_NAME}?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

#### Get name from person 
HUMAN_NAME, HUMAN_ID = friend.get_a_name_and_id(scenario, AGENT)

chat = Chat(HUMAN_ID)

#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = f"{choice(TALK_TO_ME)}"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = ""
#### Get input and loop
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    ###### Getting the next input signals
    utterance = input('\n')
    print(HUMAN_NAME + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    #### Process input and generate reply
    reply = talk.process_text_and_reply(test_triple, UtteranceType.QUESTION, scenario, place_id, location, HUMAN_ID, textSignal, chat, replier, my_brain)
    print(AGENT + ": " + reply)
    textSignal = d_util.create_text_signal(scenario, reply)
    scenario.append_signal(textSignal)


Leolani2: How's it going? What is your name? Piek?



 Piek


Piek
Leolani2: So your name is Piek?



 yes


2021-11-05 12:00:46,926 -     INFO - cltl.triple_extraction.api.Chat (Piek_t_926000)     000 - << Start of Chat with Piek_t_926000 >>
Leolani2: Would you like to chat? I'll do my best to keep up



 What is your name


Piek: What is your name
2021-11-05 12:01:34,222 -     INFO - cltl.triple_extraction.api.Chat (Piek_t_926000)     001 - Piek_t_926000: "What is your name"
UtteranceType.QUESTION
{'predicate': {'label': 'name-is', 'type': ['noun.communication', 'verb.stative']}, 'subject': {'label': 'leolani', 'type': ['agent']}, 'object': {'label': '', 'type': []}}
2021-11-05 12:01:34,228 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: leolani_like_? [agent_->_])
Leolani2: I wouldn't know!


KeyboardInterrupt: Interrupted by user

NameError: name 'answer_a_query' is not defined

In [8]:
#scenario.scenario.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

## End of notebook